# Assignment 5 - Natural Language Processing

- Student 1 - Luca Modica
- Student 2 - Hugo Alves Henriques E Silva

---

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math

sns.set_style()
%matplotlib inline

## Reading data

In [14]:
from collections import Counter
import re

# Paths to the files
de_file_path = 'dat410_europarl/europarl-v7.de-en.lc.de'
en_de_file_path = 'dat410_europarl/europarl-v7.de-en.lc.en'
fr_file_path = 'dat410_europarl/europarl-v7.fr-en.lc.fr'
en_fr_file_path = 'dat410_europarl/europarl-v7.fr-en.lc.en'
sv_file_path = 'dat410_europarl/europarl-v7.sv-en.lc.sv'
en_sv_file_path = 'dat410_europarl/europarl-v7.sv-en.lc.en'

## Warmup

In [15]:
# Function to read a file and return word frequencies
def get_word_frequencies(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read().lower()  # Ensure all text is lowercase
        words = re.findall(r'\b\w+\b', text)  # Extract words
        word_freq = Counter(words)  # Count word frequencies
    return word_freq


In [16]:
# Get word frequencies for German-English pair
de_word_freq = get_word_frequencies(de_file_path)
en_de_word_freq = get_word_frequencies(en_de_file_path)

# Print the 10 most common words in German and English (German-English pair)
de_common_words = de_word_freq.most_common(10)
en_de_common_words = en_de_word_freq.most_common(10)

# Get word frequencies for French-English pair
fr_word_freq = get_word_frequencies(fr_file_path)
en_fr_word_freq = get_word_frequencies(en_fr_file_path)

# Get word frequencies for Swedish-English pair
sv_word_freq = get_word_frequencies(sv_file_path)
en_sv_word_freq = get_word_frequencies(en_sv_file_path)

# Print the 10 most common words in French, English (French-English pair), Swedish, and English (Swedish-English pair)
fr_common_words = fr_word_freq.most_common(10)
en_fr_common_words = en_fr_word_freq.most_common(10)
sv_common_words = sv_word_freq.most_common(10)
en_sv_common_words = en_sv_word_freq.most_common(10)

print("Most common words in German:", de_common_words)
print("Most common words in English (German-English pair):", en_de_common_words)
print("Most common words in French:", fr_common_words)
print("Most common words in English (French-English pair):", en_fr_common_words)
print("Most common words in Swedish:", sv_common_words)
print("Most common words in English (Swedish-English pair):", en_sv_common_words)



Most common words in German: [('die', 10521), ('der', 9374), ('und', 7028), ('in', 4175), ('zu', 3169), ('den', 2976), ('wir', 2863), ('daß', 2738), ('ich', 2670), ('das', 2669)]
Most common words in English (German-English pair): [('the', 19853), ('of', 9633), ('to', 9069), ('and', 7307), ('in', 6278), ('is', 4478), ('that', 4441), ('a', 4438), ('we', 3372), ('this', 3362)]
Most common words in French: [('apos', 16729), ('de', 14528), ('la', 9746), ('et', 6620), ('l', 6536), ('le', 6177), ('à', 5588), ('les', 5587), ('des', 5232), ('que', 4797)]
Most common words in English (French-English pair): [('the', 19627), ('of', 9534), ('to', 8992), ('and', 7214), ('in', 6197), ('is', 4453), ('that', 4421), ('a', 4388), ('we', 3341), ('this', 3332)]
Most common words in Swedish: [('att', 9181), ('och', 7038), ('i', 5954), ('det', 5687), ('som', 5028), ('för', 4959), ('av', 4013), ('är', 3840), ('en', 3724), ('vi', 3211)]
Most common words in English (Swedish-English pair): [('the', 19327), ('o

In [17]:
# Calculate the total word counts and the counts for 'speaker' and 'zebra' across all English files
total_words = sum(en_de_word_freq.values()) + sum(en_fr_word_freq.values()) + sum(en_sv_word_freq.values())
speaker_count = en_de_word_freq['speaker'] + en_fr_word_freq['speaker'] + en_sv_word_freq['speaker']
zebra_count = en_de_word_freq['zebra'] + en_fr_word_freq['zebra'] + en_sv_word_freq['zebra']

# Calculate probabilities
prob_speaker = speaker_count / total_words
prob_zebra = zebra_count / total_words

print("Total words:", total_words)
print("Speaker count:", speaker_count)
print("Zebra count:", zebra_count)
print("Probability of 'speaker':", prob_speaker)
print("Probability of 'zebra':", prob_zebra)


Total words: 784458
Speaker count: 33
Zebra count: 0
Probability of 'speaker': 4.206726172720528e-05
Probability of 'zebra': 0.0


## Language modeling

In [18]:
from nltk.tokenize import word_tokenize
from nltk.util import bigrams

# Function to tokenize corpus into bigrams with start and end tokens
def create_bigrams(text):
    sentences = text.split('\n')
    bigram_list = []
    for sentence in sentences:
        tokens = ['<START>'] + word_tokenize(sentence) + ['<END>']
        bigram_list.extend(list(bigrams(tokens)))
    return bigram_list

# Read the English text files from all three pairs to create a single corpus
corpus_de_en = open(en_de_file_path, 'r', encoding='utf-8').read()
corpus_fr_en = open(en_fr_file_path, 'r', encoding='utf-8').read()
corpus_sv_en = open(en_sv_file_path, 'r', encoding='utf-8').read()

# Combine the corpora
combined_corpus = '\n'.join([corpus_de_en, corpus_fr_en, corpus_sv_en])

# Create bigrams from the combined corpus
bigram_list = create_bigrams(combined_corpus)

# Calculate bigram and unigram counts
unigram_counts = Counter([unigram for bigram in bigram_list for unigram in bigram])
bigram_counts = Counter(bigram_list)

# Function to calculate bigram probabilities using MLE
def calculate_bigram_probability(bigram):
    return bigram_counts[bigram] / unigram_counts[bigram[0]]

# Test the function with an example bigram
calculate_bigram_probability(('<START>', 'the')), calculate_bigram_probability(('the', 'door'))


(0.11425524114255241, 9.355332539547542e-05)

When we encounter a word that did not appear in the training texts, this will result in a probability of zero for any bigram containing this word, making the probability of the entire sentence zero. This is a common issue in language modeling known as the zero-probability problem, and it can be handled using techniques like Laplace (add-one) smoothing.

If the sentence is very long, the probability of the sentence will tend to be very small due to the multiplication of probabilities, which can lead to underflow problems in computers. One way to handle this is by working with the log probabilities instead of the raw probabilities.

In [19]:
# Calculate the vocabulary size
vocabulary_size = len(unigram_counts)

# Function to calculate bigram probabilities using Laplace smoothing
def calculate_bigram_log_probability_with_laplace(bigram):
    numerator = bigram_counts[bigram] + 1  # Add one to the count for Laplace smoothing
    denominator = unigram_counts[bigram[0]] + vocabulary_size  # Add vocabulary size for Laplace smoothing
    return math.log(numerator) - math.log(denominator)


print(calculate_bigram_log_probability_with_laplace(('<START>', 'the')), calculate_bigram_log_probability_with_laplace(('the', 'door')))

#calculate probability of a sentence
def calculate_sentence_probability(sentence):
    tokens = ['<START>'] + word_tokenize(sentence.lower()) + ['<END>']
    probability = 0
    for i in range(len(tokens) - 1):
        bigram = (tokens[i], tokens[i + 1])
        probability += calculate_bigram_log_probability_with_laplace(bigram)
    return probability

# Test the function with an example sentence
print(calculate_sentence_probability('the door is open'))
print(calculate_sentence_probability('the door is green'))
print(calculate_sentence_probability("The number of people"))

-2.503971351992991 -9.28666077584931
-39.20473330301283
-41.13676695947791
-29.65130549737112


The more negative a log probability is, the less likely the sentence is.

## Translation modeling

In [20]:
def tokenize_corpus(corpus, add_null=False):
    """Tokenize the input corpus (a list of sentences) into a list of lists of tokens.
    Optionally add a NULL token at the beginning of each sentence."""
    tokenized_corpus = [sentence.lower().split() for sentence in corpus]
    if add_null:
        for sentence in tokenized_corpus:
            sentence.insert(0, "null")
    return tokenized_corpus


def initialize_translation_prob(corpus1, corpus2):
    """Initialize translation probabilities with a lower probability for NULL."""
    unique_words_corpus1 = set(word for sentence in corpus1 for word in sentence)
    unique_words_corpus2 = set(word for sentence in corpus2 for word in sentence)
    # Calculate uniform probability excluding the NULL token
    uniform_prob = 1 / len(unique_words_corpus2 - {"NULL"})
    null_prob = uniform_prob * 0.0001  # For example, NULL token is 10 times less likely than uniform

    translation_prob = {}

    for sentence1 in corpus1:
        for sentence2 in corpus2:
            for word1 in sentence1:
                for word2 in sentence2:
                    if word2 == "null":
                        translation_prob[(word1, word2)] = null_prob
                    else:
                        translation_prob[(word1, word2)] = uniform_prob


    #print(translation_prob)

    return translation_prob


print(tokenize_corpus(["The dog runs", "The cat sleeps"]))
print(initialize_translation_prob(tokenize_corpus(["The dog runs", "The cat sleeps"]), tokenize_corpus(["Le chien court", "Le chat dort, NULL"])))

[['the', 'dog', 'runs'], ['the', 'cat', 'sleeps']]
{('the', 'le'): 0.16666666666666666, ('the', 'chien'): 0.16666666666666666, ('the', 'court'): 0.16666666666666666, ('dog', 'le'): 0.16666666666666666, ('dog', 'chien'): 0.16666666666666666, ('dog', 'court'): 0.16666666666666666, ('runs', 'le'): 0.16666666666666666, ('runs', 'chien'): 0.16666666666666666, ('runs', 'court'): 0.16666666666666666, ('the', 'chat'): 0.16666666666666666, ('the', 'dort,'): 0.16666666666666666, ('the', 'null'): 1.6666666666666667e-05, ('dog', 'chat'): 0.16666666666666666, ('dog', 'dort,'): 0.16666666666666666, ('dog', 'null'): 1.6666666666666667e-05, ('runs', 'chat'): 0.16666666666666666, ('runs', 'dort,'): 0.16666666666666666, ('runs', 'null'): 1.6666666666666667e-05, ('cat', 'le'): 0.16666666666666666, ('cat', 'chien'): 0.16666666666666666, ('cat', 'court'): 0.16666666666666666, ('sleeps', 'le'): 0.16666666666666666, ('sleeps', 'chien'): 0.16666666666666666, ('sleeps', 'court'): 0.16666666666666666, ('cat', '

In [21]:
""" def ibm_model_1(corpus_english, corpus_foreign, iterations=10):
    # Tokenize the corpora
    corpus1_tokens = tokenize_corpus(corpus_english)
    corpus2_tokens = tokenize_corpus(corpus_foreign, add_null=True)
    
    # Initialize translation probabilities uniformly
    translation_prob = initialize_translation_prob(corpus1_tokens, corpus2_tokens)

    print(translation_prob)
    
    for iteration in range(iterations):
        # Initialize counts
        count = {}
        total = {}
        for word1 in translation_prob:
            total[word1[0]] = 0
            for word2 in translation_prob:
                count[(word1[0], word2[1])] = 0
        
        # E-step: Collect counts
        for sentence1, sentence2 in zip(corpus1_tokens, corpus2_tokens):
            # Compute normalization
            #compute alignment probability for each word in sentence1
            for word1 in sentence1:
                total[word1] = sum(translation_prob[(word1, word2)] for word2 in sentence2)
            
            # update pseudo-counts using the normalized alignment probabilities
            for word1 in sentence1:
                for word2 in sentence2:
                    word_translation_prob = translation_prob[(word1, word2)]
                    count[(word1, word2)] += word_translation_prob / total[word1]
                    total[word1] += word_translation_prob / total[word1]   #pseudo-code does not include this part
        
        # M-step: Estimate probabilities
        for word1 in translation_prob:
            for word2 in translation_prob:
                if total[word1[0]] > 0:  # Avoid division by zero
                    translation_prob[(word1[0], word2[1])] = count[(word1[0], word2[1])] / total[word1[0]]
    
    return translation_prob
 """

' def ibm_model_1(corpus_english, corpus_foreign, iterations=10):\n    # Tokenize the corpora\n    corpus1_tokens = tokenize_corpus(corpus_english)\n    corpus2_tokens = tokenize_corpus(corpus_foreign, add_null=True)\n    \n    # Initialize translation probabilities uniformly\n    translation_prob = initialize_translation_prob(corpus1_tokens, corpus2_tokens)\n\n    print(translation_prob)\n    \n    for iteration in range(iterations):\n        # Initialize counts\n        count = {}\n        total = {}\n        for word1 in translation_prob:\n            total[word1[0]] = 0\n            for word2 in translation_prob:\n                count[(word1[0], word2[1])] = 0\n        \n        # E-step: Collect counts\n        for sentence1, sentence2 in zip(corpus1_tokens, corpus2_tokens):\n            # Compute normalization\n            #compute alignment probability for each word in sentence1\n            for word1 in sentence1:\n                total[word1] = sum(translation_prob[(word1, wo

In [37]:
def ibm_model_1(corpus_english, corpus_foreign, iterations=10):
    corpus1_tokens = tokenize_corpus(corpus_english)
    corpus2_tokens = tokenize_corpus(corpus_foreign, add_null=True)
    
    # Initialize translation probabilities uniformly
    translation_prob = initialize_translation_prob(corpus1_tokens, corpus2_tokens)

    print(translation_prob)
    
    for iteration in range(iterations):
        # Initialize counts
        count = {}
        total = {}
        for word1 in translation_prob:
            total[word1[0]] = 0
            for word2 in translation_prob:
                count[(word1[0], word2[1])] = 0
        
        # E-step: Collect counts
        for sentence1, sentence2 in zip(corpus1_tokens, corpus2_tokens):
            # Compute normalization
            #compute alignment probability for each word in sentence1
            for word1 in sentence1:
                total[word1] = sum(translation_prob[(word1, word2)] for word2 in sentence2)
            
            # update pseudo-counts using the normalized alignment probabilities
            for word1 in sentence1:
                for word2 in sentence2:
                    word_translation_prob = translation_prob[(word1, word2)]
                    count[(word1, word2)] += word_translation_prob / total[word1]
                    total[word1] += word_translation_prob / total[word1]   #pseudo-code does not include this part
        
        # M-step: Estimate probabilities
        for word1 in translation_prob:
            for word2 in translation_prob:
                if total[word1[0]] > 0:  # Avoid division by zero
                    translation_prob[(word1[0], word2[1])] = count[(word1[0], word2[1])] / total[word1[0]]
    
    return translation_prob


In [38]:
# Example usage (using dummy data):
corpus1 = ["the house", "the book", "a big house"]
corpus2 = ["das haus", "das buch", "ein großes haus"]  # Assuming German for demonstration

# Estimate translation probabilities
translation_prob = ibm_model_1(corpus1, corpus2, iterations=10)

# Find translations for a specific word (e.g., "house")
translations_for_word = {pair[1]: prob for pair, prob in translation_prob.items() if pair[0] == "house"}
# Sort translations by probability
sorted_translations = sorted(translations_for_word.items(), key=lambda item: item[1], reverse=True)

# Print top N translations
print("Top translations for 'house':")
for foreign_word, prob in sorted_translations[:10]:
    print(f"{foreign_word}: {prob}")

Top translations for 'house':
haus: 0.9972041747744738
buch: 0.16666666666666666
das: 0.0012866134176835547
ein: 0.0007047456951826293
großes: 0.0007047456951826293
null: 9.972041747744736e-05


Need to be very careful with the NULL probability. It needs to be way lower than the other probabilities, otherwise the model will always choose the NULL translation, because NULL will be present in every sentence. Increasing the number of iterations will eventually make null the most probable translation for every word.

In [36]:
#Write code that implements the estimation algorithm for IBM model 1.
# Then print, for either Swedish, German, or French, the 10 words that 
#the English word european is most likely to be translated into, according 
#to your estimate. It can be interesting to look at this list of 10 words and
#see how it changes during the EM iterations.

#reduce the size of corpus_de_en and corpus_sv_en
sv_en = open(en_sv_file_path, 'r', encoding='utf-8').read()
#separate the sentences
sv_en = sv_en.split("\n")
corpus_en = sv_en[:5]

sv = open(sv_file_path, 'r', encoding='utf-8').read()
#separate the sentences
sv = sv.split("\n")
corpus_sv = sv[:5]

#print(corpus_en)
#print(corpus_sv)

# Estimate translation probabilities
translation_prob = ibm_model_1(corpus_en, corpus_sv, iterations=1)

# Find translations for a specific word (e.g., "european")
translations_for_word = {pair[1]: prob for pair, prob in translation_prob.items() if pair[0] == "european"}
# Sort the top 10 translations by probability
swedish_translations = sorted(translations_for_word.items(), key=lambda item: item[1], reverse=True)[:10]

print("Top translations for 'european' in Swedish:")
for foreign_word, prob in swedish_translations:
    print(f"{foreign_word}: {prob}")



{('i', 'null'): 1.3333333333333334e-06, ('i', 'jag'): 0.013333333333333334, ('i', 'förklarar'): 0.013333333333333334, ('i', 'europaparlamentets'): 0.013333333333333334, ('i', 'session'): 0.013333333333333334, ('i', 'återupptagen'): 0.013333333333333334, ('i', 'efter'): 0.013333333333333334, ('i', 'avbrottet'): 0.013333333333333334, ('i', 'den'): 0.013333333333333334, ('i', '17'): 0.013333333333333334, ('i', 'december'): 0.013333333333333334, ('i', '.'): 0.013333333333333334, ('i', 'vill'): 0.013333333333333334, ('i', 'på'): 0.013333333333333334, ('i', 'nytt'): 0.013333333333333334, ('i', 'önska'): 0.013333333333333334, ('i', 'er'): 0.013333333333333334, ('i', 'ett'): 0.013333333333333334, ('i', 'gott'): 0.013333333333333334, ('i', 'år'): 0.013333333333333334, ('i', 'och'): 0.013333333333333334, ('i', 'hoppas'): 0.013333333333333334, ('i', 'att'): 0.013333333333333334, ('i', 'ni'): 0.013333333333333334, ('i', 'haft'): 0.013333333333333334, ('i', 'en'): 0.013333333333333334, ('i', 'trevl

## Decoding